In [7]:
%load_ext autoreload
%autoreload 2

import util as yu
from util import *

yu.setpath('check_sensitivity_c3pt_to_c2pt')

projs=['P0', 'Px', 'Py', 'Pz']
inserts=['tt', 'tx', 'ty', 'tz', 'xx', 'xy', 'xz', 'yy', 'yz', 'zz']
enss=['b','c','d']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
path='pkl/analysis_c2pt/reg_ignore/ens2pars_jk_meffnst_selected.pkl'
[ens2pars_jk_meff1st,ens2pars_jk_meff2st,ens2pars_jk_meff3st]=yu.load_pkl(path)

key2tf2ratio={}
for ens in enss:
    key2tf2ratio[(ens,'j+;conn')]={}
    key2tf2ratio[(ens,'j-;conn')]={}
    
    basepath=f'/p/project1/ngff/li47/code/projectData/05_moments/{yu.ens2full[ens]}/data_merge/'
    
    mN_jk=ens2pars_jk_meff2st[ens][:,0]
    factor_equal=1/(-3*mN_jk/4)
    # factor_equal=np.mean(factor_equal)[None]
    
    path=f'{basepath}/conn_2pt.h5'
    with h5py.File(path) as f:
        moms=yu.moms2list(f['moms'])
        imom=moms.index([0,0,0])
        
        tf2c2pt={}
        for tf in f['data'].keys():
            t=f[f'data/{tf}'][:]
            t=yu.jackknife(np.real(t[:,:,imom]))
            tf2c2pt[int(tf)]=t

    path=f'{basepath}/conn_0,0,0,0,0,0.h5'
    with h5py.File(path) as f:
        moms=yu.moms2list(f['moms'])
        imom=moms.index([0,0,0,0,0,0])
        
        for jtf in f['data'].keys():
            j,tf=jtf.split('_'); tf=int(tf)
            t=f[f'data/{jtf}'][:]
            t=t[:,:,0,projs.index('P0'),inserts.index('tt')]
            c3pt=yu.jackknife(t)
            ratio=np.real(c3pt/tf2c2pt[tf][:,tf:tf+1]*factor_equal[:,None])
            key=(ens,j)
            key2tf2ratio[key][tf]=ratio
            
ens2tfs_conn={}
for ens in enss:
    tfs=list(key2tf2ratio[(ens,'j+;conn')].keys()); tfs.sort()
    ens2tfs_conn[ens]=tfs
    print(ens,tfs)

b [8, 10, 12, 14, 16, 18, 20]
c [6, 8, 10, 12, 14, 16, 18, 20, 22]
d [8, 10, 12, 14, 16, 18, 20, 22, 24, 26]


In [9]:
keys=[(ens,j) for j in ['j+;conn','j-;conn'] for ens in enss]
key2fits={}

tftcmins=[(14,3),(10,3),(14,4),(12,3),(14,4),(8,3)]

def modify(pars_jk_meff2st,i,shift=0):
    t=pars_jk_meff2st.copy()
    mean,err=yu.jackme(t)
    t[:,i]=t[:,i]*0+mean[i]+shift*err[i]
    return t
funcs=[lambda x:x,lambda x:modify(x,0),lambda x:modify(x,1),lambda x:modify(x,2),lambda x:modify(modify(modify(x,0),1),2)]
columns=['normal','0err_m','0err_dE','0err_rc1','0err']

results=[]
for ifunc,func in enumerate(funcs):
    for i,key in enumerate(keys):
        print(f'{ifunc}/{len(funcs)}',f'{i}/{len(keys)}',key,end='               \r')
        ens,j=key
        tf2ratio=key2tf2ratio[key]
        
        pars_jk_meff2st=ens2pars_jk_meff2st[ens]
        tfmins,tcmins=[tftcmins[i][0]],[tftcmins[i][1]]
        fits=yu.doFit_3ptSym_2st2step(tf2ratio,tfmins,tcmins,func(pars_jk_meff2st),downSampling=[1,1],symmetrizeQ=False)
        key2fits[key]=fits
        
    list_tf2ratio_fits=[[key2tf2ratio[key],key2fits[key],func(ens2pars_jk_meff2st[key[0]])] for key in keys]
    fig,axs,result=yu.makePlot_3pt_ChristosStyle(list_tf2ratio_fits,xunit=yu.ens2a[ens],tcmin_rainbow=2)
    for i,key in enumerate(keys):
        axs[i,0].set_ylabel(key)
    yu.finalizePlot(closeQ=True)
    
    results.append(result)

tab=[[yu.jackme_un2str(results[j][i][:,0]) for j in range(len(columns))] for i,key in enumerate(keys)]
tab=pd.DataFrame(tab,index=[f'{j}_{yu.ens2label[ens]}' for ens,j in keys],columns=columns)

display(tab)

,normal,0err_m,0err_dE,0err_rc1,0err
j+;conn_B64,0.386(15),0.386(15),0.3860(91),0.386(15),0.3860(91)
j+;conn_C80,0.329(17),0.329(17),0.3285(53),0.329(17),0.3285(53)
j+;conn_D96,0.327(16),0.327(16),0.3271(39),0.327(16),0.3271(39)
j-;conn_B64,0.1597(90),0.1597(90),0.1597(80),0.1597(90),0.1597(80)
j-;conn_C80,0.1405(73),0.1405(73),0.1405(59),0.1405(73),0.1405(59)
j-;conn_D96,0.1338(74),0.1338(74),0.1338(28),0.1338(74),0.1338(28)


In [10]:
keys=[(ens,j) for j in ['j+;conn','j-;conn'] for ens in enss]
key2fits={}

tftcmins=[(14,3),(10,3),(14,4),(12,3),(14,4),(8,3)]

def modify(pars_jk_meff2st,i,shift=0):
    t=pars_jk_meff2st.copy()
    mean,err=yu.jackme(t)
    # t[:,i]=t[:,i]*0+mean[i]+shift*err[i]
    t[:,i]=t[:,i]+shift*err[i]
    return t
funcs=[lambda x:x,lambda x:modify(x,1,shift=-1),lambda x:modify(x,1,shift=0),lambda x:modify(x,1,shift=+1)]
columns=['normal','dE-1','dE','dE+1']

results=[]
for ifunc,func in enumerate(funcs):
    for i,key in enumerate(keys):
        print(f'{ifunc}/{len(funcs)}',f'{i}/{len(keys)}',key,end='               \r')
        ens,j=key
        tf2ratio=key2tf2ratio[key]
        
        pars_jk_meff2st=ens2pars_jk_meff2st[ens]
        tfmins,tcmins=[tftcmins[i][0]],[tftcmins[i][1]]
        fits=yu.doFit_3ptSym_2st2step(tf2ratio,tfmins,tcmins,func(pars_jk_meff2st),downSampling=[1,1],symmetrizeQ=False)
        key2fits[key]=fits
        
    list_tf2ratio_fits=[[key2tf2ratio[key],key2fits[key],func(ens2pars_jk_meff2st[key[0]])] for key in keys]
    fig,axs,result=yu.makePlot_3pt_ChristosStyle(list_tf2ratio_fits,xunit=yu.ens2a[ens],tcmin_rainbow=2)
    for i,key in enumerate(keys):
        axs[i,0].set_ylabel(key)
    yu.finalizePlot(closeQ=True)
    
    results.append(result)

tab=[[yu.jackme_un2str(results[j][i][:,0]) for j in range(len(columns))] for i,key in enumerate(keys)]
tab=pd.DataFrame(tab,index=[f'{j}_{yu.ens2label[ens]}' for ens,j in keys],columns=columns)

display(tab)

,normal,dE-1,dE,dE+1
j+;conn_B64,0.386(15),0.373(17),0.386(15),0.397(13)
j+;conn_C80,0.329(17),0.309(21),0.329(17),0.344(14)
j+;conn_D96,0.327(16),0.310(19),0.327(16),0.341(13)
j-;conn_B64,0.1597(90),0.154(10),0.1597(90),0.1642(82)
j-;conn_C80,0.1405(73),0.1349(85),0.1405(73),0.1451(65)
j-;conn_D96,0.1338(74),0.1262(88),0.1338(74),0.1401(63)


In [11]:
keys=[(ens,j) for j in ['j+;conn','j-;conn'] for ens in enss]
key2fits={}

tftcmins=[(14,3),(10,3),(14,4),(12,3),(14,4),(8,3)]

def modify(pars_jk_meff2st,i,shift=0):
    t=pars_jk_meff2st.copy()
    mean,err=yu.jackme(t)
    # t[:,i]=t[:,i]*0+mean[i]+shift*err[i]
    t[:,i]=t[:,i]+shift*err[i]
    return t
funcs=[lambda x:x,lambda x:modify(x,0,shift=-10),lambda x:modify(x,0,shift=0),lambda x:modify(x,0,shift=+10)]
columns=['normal','m-10','m','m+10']

results=[]
for ifunc,func in enumerate(funcs):
    for i,key in enumerate(keys):
        print(f'{ifunc}/{len(funcs)}',f'{i}/{len(keys)}',key,end='               \r')
        ens,j=key
        tf2ratio=key2tf2ratio[key]
        
        pars_jk_meff2st=ens2pars_jk_meff2st[ens]
        tfmins,tcmins=[tftcmins[i][0]],[tftcmins[i][1]]
        fits=yu.doFit_3ptSym_2st2step(tf2ratio,tfmins,tcmins,func(pars_jk_meff2st),downSampling=[1,1],symmetrizeQ=False)
        key2fits[key]=fits
        
    list_tf2ratio_fits=[[key2tf2ratio[key],key2fits[key],func(ens2pars_jk_meff2st[key[0]])] for key in keys]
    fig,axs,result=yu.makePlot_3pt_ChristosStyle(list_tf2ratio_fits,xunit=yu.ens2a[ens],tcmin_rainbow=2)
    for i,key in enumerate(keys):
        axs[i,0].set_ylabel(key)
    yu.finalizePlot(closeQ=True)
    
    results.append(result)

tab=[[yu.jackme_un2str(results[j][i][:,0]) for j in range(len(columns))] for i,key in enumerate(keys)]
tab=pd.DataFrame(tab,index=[f'{j}_{yu.ens2label[ens]}' for ens,j in keys],columns=columns)

display(tab)

# the parameterization of the ratio is actually independent of m

,normal,m-10,m,m+10
j+;conn_B64,0.386(15),0.386(15),0.386(15),0.386(15)
j+;conn_C80,0.329(17),0.329(17),0.329(17),0.329(17)
j+;conn_D96,0.327(16),0.327(16),0.327(16),0.327(16)
j-;conn_B64,0.1597(90),0.1597(90),0.1597(90),0.1597(90)
j-;conn_C80,0.1405(73),0.1405(73),0.1405(73),0.1405(73)
j-;conn_D96,0.1338(74),0.1338(74),0.1338(74),0.1338(74)
